In [1]:
import pandas as pd
import altair as alt

dataframe=pd.read_csv("https://raw.githubusercontent.com/jfrausig/test/master/cardata_sample_v2.csv", sep=';')


import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

import altair as alt
import pandas as pd

a=alt.binding_range(min=-10, max=10, step=0.01,name="a")
b=alt.binding_range(min=-10, max=10, step=0.01,name="b")
c=alt.binding_range(min=-10, max=10, step=0.01,name="c")
d=alt.binding_range(min=-1000, max=1000, step=0.01,name="d")

Min_value=alt.binding_range(min=-1, max=10, step=0.1,name="Min_value")

Max_value=alt.binding_range(min=-1, max=200, step=0.1,name="Max_value")


data=pd.DataFrame()
#data['x_1']=list(range(1,15))
#data['x_2']=list(range(5,19))

data['x_1']=dataframe['accelerator_pedal_position']
data['y_1']=dataframe['torque_at_transmission']


regression=alt.Chart(data).transform_fold(
    ['y_1','f_x_1']
).mark_line().encode(
    x='x_1:Q',
    y='value:Q',
    color='key:N'
).properties(height=100)

diff=alt.Chart(data).mark_point().encode(
   x='x_1:Q',
   y='diff:Q',
).properties(height=100, title="differense mellem f_x og y_1")


diff_2nd=alt.Chart(data).mark_point().encode(
   x='x_1:Q',
   y='diff_2nd:Q',

).properties(height=100)

R_1=alt.Chart(data).mark_text(size=16).encode(
  text='r_1:Q'

).properties(height=100, title="differense mellem f_x og y_1 i anden potens")

auto_linear=alt.Chart(data).transform_regression('x_1', 'y_1').mark_point(color="red").encode(x='x_1:Q',y='y_1:Q')

alt.vconcat(regression+auto_linear,diff,diff_2nd+R_1).transform_filter(
    '(datum.x_1>Min_value.Min_value)&&(datum.x_1<Max_value.Max_value)'
).transform_calculate(
    x_x='datum.x_1',
    y_y='datum.y_1',
    x_1_sqrd='pow(datum.x_1,2)',
    x_1_y_1='datum.x_1*datum.y_1',
    y_1_sqrd='pow(datum.y_1,2)',
).transform_window(
    sum_x_1_sqrd='sum(x_1_sqrd)',
    sum_x_1_y_1='sum(x_1_y_1)',
    sum_x_1='sum(x_1)',
    sum_y_1='sum(y_1)',
    N='count(x_1)',
    sum_y_1_sqrd='sum(y_1_sqrd)',
    frame=[None,None]
).transform_calculate(
    m='(datum.N*datum.sum_x_1_y_1-datum.sum_x_1*datum.sum_y_1)/(datum.N*datum.sum_x_1_sqrd-datum.sum_x_1*datum.sum_x_1)',
    b='(datum.sum_y_1-datum.m*datum.sum_x_1)/datum.N',
    f_x_1='datum.m*datum.x_1+datum.b',
    diff='datum.y_1-datum.f_x_1',
    diff_2nd='pow(datum.y_1-datum.f_x_1,2)',
    r_1='(datum.N*datum.sum_x_1_y_1-datum.sum_x_1*datum.sum_y_1)/(sqrt(datum.N*datum.sum_x_1_sqrd-datum.sum_x_1*datum.sum_x_1)*sqrt(datum.N*datum.sum_y_1_sqrd-pow(datum.sum_y_1,2)))'


).add_selection(
    alt.selection_single(fields=['Min_value'], bind=Min_value , name='Min_value', init={"Min_value":'2'}),
    alt.selection_single(fields=['Max_value'], bind=Max_value , name='Max_value', init={"Max_value":'90'}),
).properties(title="linear regression")





alt.VConcatChart(...)